In [1]:
using CairoMakie, Dierckx, Optim, LinearAlgebra, QuantEcon

In [29]:
pars = (;α = 0.33, # Capital share
        β = 0.9, # Discount factor
        A = 10.0, # TFP
        γ = 1.0, # Risk aversion
        δ = 0.1, # Depreciation rate
        nk = 31, # Number of capital gridpoints
        θ = 4, # Grid expansion parameter
        lb = 0.00, # Lower bound of capital grid
        ub = 1000.0, # Upper bound of capital grid
        nz = 19, # Number of shock gridpoints
        ρ = 0.9, # Persistence of AR(1) process
        μ = 0.0, # Mean of AR(1) process
        σ = 0.007, # Var of AR(1) process
        toler = 1e-6, # Tolerance
        maxiter = 10000) # Maximum number of iterations

(α = 0.33, β = 0.9, A = 10.0, γ = 1.0, δ = 0.1, nk = 31, θ = 4, lb = 0.0, ub = 1000.0, nz = 19, ρ = 0.9, μ = 0.0, σ = 0.007, toler = 1.0e-6, maxiter = 10000)

In [30]:
function utility(c)
    if pars.γ == 1.0
        return log(c)
    else
        return (c^(1-pars.γ)) / (1-pars.γ)
    end
end

function ar1(pars)
    (; μ, ρ, σ, nz) = pars
    mc = rouwenhorst(nz, μ, ρ, σ)
    Π, Zvals = mc.p, exp.(mc.state_values)
    return Π, Zvals
end

function exp_grid(pars)
    (; nk, θ, lb, ub) = pars
    grid = LinRange(1e-4, 1.0, nk)
    expgrid = lb .+ (ub .- lb) .* grid.^θ
    return expgrid
end

function interpolate(grid, vals, pars)
    spline = Spline1D(grid, vals, k = 1, bc = "extrapolate")
    return spline
end

function production(k, z ,pars)
    (; α, δ, A) = pars
    return A * z * (k ^ α) + (1.0 - δ) * k
end

function initial_guess(grid, pars)
    (; nk, nz) = pars
    v = ones(nk, nz)
    v_out = zeros(nk, nz)
    for j in 1:nz
        v_out[:,j] = utility.(grid)
    end
    return v_out
end

function resource_grid(kgrid, zgrid, pars)
    (; nk, nz) = pars
    Ygrid = zeros(nk, nz)
    for i in 1:nk
        for j in 1:nz
            Ygrid[i,j] = production(kgrid[i], zgrid[j], pars)
        end
    end
    return Ygrid
end

resource_grid (generic function with 1 method)

In [32]:
function egm(pars)
    (; γ, nk, nz, toler, maxiter) = pars
    Π, Zvals = ar1(pars)
    Kvals = exp_grid(pars)
    v = initial_guess(exp_grid(pars), pars)
    v_new = zeros(nk, nz)
    c = zeros(nk, nz)
    derivatives = zeros(nk, nz)
    Yvals = resource_grid(Kvals, Zvals, pars)
    error = toler + 1
    iter = 0
    if iter == 0
        println("Iterating...")
    end
    while ((error > toler) && (iter < maxiter))
        interpolators = [interpolate(Kvals, v[:,j], pars) for j in 1:nz]
        for (j, interpolator) in enumerate(interpolators)
            derivatives[:,j] = Dierckx.derivative(interpolator, Kvals)
        end
        c = (derivatives).^(-1/γ)
        error = 0
    end
    return c
end
test = egm(pars)  

31×19 Matrix{Float64}:
   5.37226e-5    5.37226e-5    5.37226e-5  …    5.37226e-5    5.37226e-5
   0.00672847    0.00672847    0.00672847       0.00672847    0.00672847
   0.0496934     0.0496934     0.0496934        0.0496934     0.0496934
   0.188315      0.188315      0.188315         0.188315      0.188315
   0.511536      0.511536      0.511536         0.511536      0.511536
   1.13792       1.13792       1.13792     …    1.13792       1.13792
   2.21563       2.21563       2.21563          2.21563       2.21563
   3.92249       3.92249       3.92249          3.92249       3.92249
   6.46588       6.46588       6.46588          6.46588       6.46588
  10.0829       10.0829       10.0829          10.0829       10.0829
   ⋮                                       ⋱                
 316.553       316.553       316.553          316.553       316.553
 376.674       376.674       376.674          376.674       376.674
 444.978       444.978       444.978          444.978       444.978
 52